In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans

import sys
sys.path.insert(0, '../Modules/')
sys.path.insert(0, '../Shared code/')
/
import pose_estimation as pe  # noqa (ignore PEP 8 style)
from gait_metrics import gait_dataframe  # noqa
from peakdet import peakdet  # noqa
from general import mad_outliers

# %% Read DataFrame

directory = '../../../MEGA/Data/Kinect Zeno/Kinect processed'
file_name = '2014-12-22_P007_Pre_004.pkl'

load_path = os.path.join(directory, file_name)
df = pd.read_pickle(load_path)


# %% Parameters

lower_part_types = ['HEAD', 'HIP', 'UPPER_LEG', 'KNEE', 'LOWER_LEG', 'FOOT']


lengths = [62.1080, 20.1733, 14.1756, 19.4509, 20.4996]

radii = [i for i in range(0, 30, 5)]

edges = np.matrix('0 1; 1 2; 2 3; 3 4; 4 5; 3 5; 1 3')


# %% Select positions

# Find the best positions for each image frame
best_pos_series = df.apply(lambda x: pe.process_frame(x.to_dict(),
                           lower_part_types, edges, lengths, radii), axis=1)

n_frames = len(df)
for f in range(n_frames):
    row = df.loc[f]

    pe.process_frame(row.to_dict(), lower_part_types, edges, lengths, radii)


# %% Extract head and foot positions

# Each row i is a tuple containing the best positions for frame i
# Split each tuple into columns of a DataFrame
df_best_pos = pd.DataFrame(best_pos_series.values.tolist(),
                           columns=['Side A', 'Side B'])

df_best_pos = df_best_pos.dropna()

# Extract the head and feet positions
head_pos = df_best_pos['Side A'].apply(lambda row: row[0, :])
L_foot_pos = df_best_pos['Side A'].apply(lambda row: row[-1, :])
R_foot_pos = df_best_pos['Side B'].apply(lambda row: row[-1, :])

# Combine into new DataFrame
df_head_feet = pd.concat([head_pos, L_foot_pos, R_foot_pos], axis=1)
df_head_feet.columns = ['HEAD', 'L_FOOT', 'R_FOOT']
df_head_feet.index.name = 'Frame'


# %% Remove outlier frames

y_foot_L = df_head_feet.apply(lambda row: row['L_FOOT'][1], axis=1).values
y_foot_R = df_head_feet.apply(lambda row: row['R_FOOT'][1], axis=1).values

y_foot_L_filtered = mad_outliers(y_foot_L, 2)
y_foot_R_filtered = mad_outliers(y_foot_R, 2)

good_frame_L = ~np.isnan(y_foot_L_filtered)
good_frame_R = ~np.isnan(y_foot_R_filtered)

df_head_feet = df_head_feet[good_frame_L & good_frame_R]


# %% Gait metrics


foot_dist = df_head_feet.apply(lambda row: np.linalg.norm(
                               row['L_FOOT'] - row['R_FOOT']), axis=1)


In [154]:
# Cluster frames with k means to locate the 4 walking passes
frames = foot_dist.index.values.reshape(-1, 1)
k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

In [156]:
frame_labels = k_means.labels_


In [158]:
group_0 = frame_labels==0

In [250]:
# All positions on one walking pass
head_points = np.stack(tuple(df_head_feet['HEAD'][group_0]))
L_foot_points = np.stack(tuple(df_head_feet['L_FOOT'][group_0]))
R_foot_points = np.stack(tuple(df_head_feet['R_FOOT'][group_0]))

mean_foot_points = (L_foot_points + R_foot_points) / 2

In [293]:
centroid, direction = best_fit_line(head_points)

In [311]:
from linear_algebra import proj_point_plane

# Project all head points onto the XZ plane
plane_point = np.array([0, 0, 0])
normal = np.array([0, 1, 0])

head_points_proj = np.apply_along_axis(proj_point_plane, 1, head_points, plane_point, normal)

L_foot_points_proj = np.apply_along_axis(proj_point_plane, 1, L_foot_points,
                                         plane_point, normal)
R_foot_points_proj = np.apply_along_axis(proj_point_plane, 1, R_foot_points,
                                         plane_point, normal)


In [365]:
from linear_algebra import angle_direction, dist_point_line

def assign_LR(foot_A, foot_B, line_vector):
    
    # Up direction defined as positive y
    up = np.array([0, 1, 0])  
    
    # Points on line
    line_point_A = (foot_A + foot_B) / 2
    line_point_B = line_point_A + line_vector
          
    # Vector from mean foot position to current foot
    target_direction = foot_A - line_point_A

    # Check if point is left or right of the line
    return angle_direction(target_direction, line_vector, up)
        
        

#### Putting it together

In [430]:
def consistent_sides(df_head_feet, frame_labels):
    """
    
    Parameters
    ----------
    df_head_feet : DataFrame
    
    frame_labels : array_like
    
    Returns
    -------
    switch_sides : Series
    
    """
    
    frames = df_head_feet.index.values
    
    n_frames = len(frame_labels)
    n_labels = frame_labels.max() + 1
    
    switch_sides = pd.Series(np.full(n_frames, False), index=frames)
    
    for i in range(n_labels):

        cluster_i = frame_labels == i

        # All positions on one walking pass
        head_points = np.stack(tuple(df_head_feet['HEAD'][cluster_i]))

        # Line of best fit for all head positions
        centroid, direction = best_fit_line(head_points)

        cluster_frames = frames[frame_labels == i]
        
        for frame in cluster_frames:
            
            foot_A = df_head_feet.loc[frame, 'L_FOOT']
            foot_B = df_head_feet.loc[frame, 'R_FOOT']

            side_of_A = assign_LR(foot_A, foot_B, direction)

            if side_of_A == 1:
                # A value of 1 indicates that foot A is on the right of the line of best fit
                # Thus, the sides should be switched
                switch_sides.loc[frame] = True
            
    return switch_sides

In [438]:
switch_sides = consistent_sides(df_head_feet, frame_labels)

In [439]:
colors = ['b', 'r']

frames = df_head_feet.index.values

for frame in range(600, 620):

    points_A = df_best_pos.loc[frame].iloc[0]
    points_B = df_best_pos.loc[frame].iloc[1]
        
    if switch_sides[frame]:
        points_A, points_B = points_B, points_A
    
    fig = plt.figure()
    scatter_pos(fig, [points_A, points_B], colors)
    plt.title('Frame {}'.format(frame))

#### Switching the DataFrame

In [460]:
df_head_feet_final = df_head_feet.copy()

In [464]:
for frame, switch in switch_sides.iteritems():
    
    if switch:
        df_head_feet_final.loc[frame, ['L_FOOT', 'R_FOOT']] = df_head_feet_final.loc[frame, ['R_FOOT', 'L_FOOT']].values

In [465]:
df_head_feet_final.loc[244]

HEAD                    [-116.4, 65.5421, 249.058]
L_FOOT    [-101.86200000000001, -54.4605, 270.199]
R_FOOT               [-101.744, -54.7345, 270.191]
Name: 244, dtype: object

In [466]:
df_head_feet.loc[244]

HEAD                    [-116.4, 65.5421, 249.058]
L_FOOT               [-101.744, -54.7345, 270.191]
R_FOOT    [-101.86200000000001, -54.4605, 270.199]
Name: 244, dtype: object